In [ ]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim #similarity between two images based on luminance, contrast, and structure.
from concurrent.futures import ThreadPoolExecutor

def is_duplicate(frame, previous_frames, threshold=0.95):
    for prev_frame in previous_frames:
        ssim_index, _ = ssim(frame, prev_frame, multichannel=True, full=True)
        if ssim_index > threshold:
            return True
    return False

def is_blank(frame, threshold=10):
    mean_pixel_value = np.mean(frame)
    return mean_pixel_value < threshold

def process_frame(args):
    i, frame, previous_frames, output_folder = args
    if is_duplicate(frame, previous_frames):
        print(f"Skipping duplicate frame {i}")
        return None

    if is_blank(frame):
        print(f"Skipping blank frame {i}")
        return None

    previous_frames.append(frame.copy())
    if len(previous_frames) > 5:
        previous_frames.pop(0)

    frame_filename = os.path.join(output_folder, f"frame_{i:04d}.jpg")
    cv2.imwrite(frame_filename, frame)
    return frame_filename

def extract_frames(video_path, output_folder):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    previous_frames = []

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    args_list = [(i, cap.read()[1], previous_frames.copy(), output_folder) for i in range(frame_count)]
    with ThreadPoolExecutor() as executor:
        result_filenames = list(executor.map(process_frame, args_list))

    cap.release()

# Example usage:
video_path = "/content/drive/MyDrive/UnStop/HP_short"
output_folder = "/content/drive/MyDrive/UnStop/Frames"
extract_frames(video_path, output_folder)

Skipping blank frame 0Skipping blank frame 1
Skipping blank frame 2

Skipping blank frame 3
Skipping blank frame 4
Skipping blank frame 5
Skipping blank frame 6
Skipping blank frame 8
Skipping blank frame 7
Skipping blank frame 9
Skipping blank frame 10Skipping blank frame 11

Skipping blank frame 33
Skipping blank frame 34
Skipping blank frame 58
Skipping blank frame 59
Skipping blank frame 60
Skipping blank frame 61
Skipping blank frame 62
Skipping blank frame 63
Skipping blank frame 64
Skipping blank frame 65
Skipping blank frame 67
Skipping blank frame 66
Skipping blank frame 68
Skipping blank frame 72
Skipping blank frame 73
Skipping blank frame 74
Skipping blank frame 88
Skipping blank frame 89
Skipping blank frame 90
Skipping blank frame 91
Skipping blank frame 96
Skipping blank frame 97
Skipping blank frame 109
Skipping blank frame 110
Skipping blank frame 111
Skipping blank frame 112
Skipping blank frame 113
Skipping blank frame 226
Skipping blank frame 227
Skipping blank fram

In [ ]:
from transformers import CLIPProcessor, CLIPModel
import torch
import cv2
import numpy as np
import os

# Load CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

# Function to classify frames in batches
def classify_frames_batch(frames, categories, batch_size=16):
    probs = []
    for i in range(0, len(frames), batch_size):
        batch_frames = frames[i:i + batch_size]
        inputs = processor(text=categories, images=batch_frames, return_tensors="pt", padding=True)
        outputs = model(**inputs)
        logits_per_image = outputs.logits_per_image  # Image features
        probs.extend(logits_per_image.softmax(dim=1).tolist())  # Probability distribution over labels
    return torch.tensor(probs)

# Function to create video from frames
def create_video_from_frames(frames, output_path, fps):
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for frame in frames:
        out.write(frame)

    out.release()

# Example usage
video_frames_folder = '/content/drive/MyDrive/UnStop/Frames'
output_video_path = '/content/drive/MyDrive/UnStop/Teaser.mp4'

# Define multiple categories suitable for a video trailer
categories = ["Sad", "Happy", "Exciting", "Mysterious"]

# Collect frames from the video_frames_folder
video_frames = []
for filename in sorted(os.listdir(video_frames_folder)):
    frame_path = os.path.join(video_frames_folder, filename)
    frame = cv2.imread(frame_path)
    video_frames.append(frame)

# Classify frames in batches
probs = classify_frames_batch(video_frames, categories)

# Display the probabilities for each category
for i, category in enumerate(categories):
    print(f"Probability for {category}: {probs[:, i].mean().item()}")

# Filter frames based on the chosen categories and a probability threshold
threshold = 0.7
chosen_frames = [frame for frame, prob in zip(video_frames, probs) if any(p > threshold for p in prob)]

# Create a video using the chosen frames
create_video_from_frames(chosen_frames, output_video_path, fps=30)  # Assuming original video FPS is 30


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/599M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Probability for Sad: 0.3731710612773895
Probability for Happy: 0.2716536223888397
Probability for Exciting: 0.2010461539030075
Probability for Mysterious: 0.15412920713424683


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import CLIPProcessor, CLIPModel
import torch
import cv2
import numpy as np
import os

# Load CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

# Function to classify frames in batches
def classify_frames_batch(frames, categories, batch_size=16):
    probs = []
    for i in range(0, len(frames), batch_size):
        batch_frames = frames[i:i + batch_size]
        inputs = processor(text=categories, images=batch_frames, return_tensors="pt", padding=True)
        outputs = model(**inputs)
        logits_per_image = outputs.logits_per_image  # Image features
        probs.extend(logits_per_image.softmax(dim=1).tolist())  # Probability distribution over labels
    return torch.tensor(probs)

# Function to create video from frames
def create_video_from_frames(frames, output_path, fps):
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for frame in frames:
        out.write(frame)

    out.release()

# Example usage
video_frames_folder = '/content/drive/MyDrive/UnStop/Frames'
output_folder = '/content/drive/MyDrive/UnStop/output_videos'

# Define multiple categories suitable for a video trailer
categories = ["Sad", "Happy", "Exciting", "Mysterious"]

# Collect frames from the video_frames_folder
video_frames = []
for filename in sorted(os.listdir(video_frames_folder)):
    frame_path = os.path.join(video_frames_folder, filename)
    frame = cv2.imread(frame_path)
    video_frames.append(frame)

# Classify frames in batches
probs = classify_frames_batch(video_frames, categories)

# Display the probabilities for each category
for i, category in enumerate(categories):
    avg_prob = probs[:, i].mean().item()
    print(f"Probability for {category}: {avg_prob}")
    category_frames = [frame for frame, prob in zip(video_frames, probs) if prob[i] > avg_prob]
    create_video_from_frames(category_frames, os.path.join(output_folder, f'{category}_frames.mp4'), fps=30)

# Choose the category for the final mixed video
# ...

# Choose the category for the final mixed video
chosen_category_index = int(input("Enter the index of the category for the final mixed video: "))
chosen_category_frames = [frame for frame, prob in zip(video_frames, probs) if prob[chosen_category_index] > 0.7]

# Check if there are frames in the chosen category
if len(chosen_category_frames) > 0:
    # Create a video using the chosen frames
    create_video_from_frames(chosen_category_frames, os.path.join(output_folder, 'final_mixed_video.mp4'), fps=30)
    print("Final mixed video created successfully.")
else:
    print("No frames found for the chosen category. Adjust the threshold or choose another category.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Probability for Sad: 0.3731710612773895
Probability for Happy: 0.2716536223888397
Probability for Exciting: 0.2010461539030075
Probability for Mysterious: 0.15412920713424683
Enter the index of the category for the final mixed video: 1
Final mixed video created successfully.


In [ ]:
pip install moviepy


In [ ]:
# from moviepy.editor import VideoFileClip,AudioFileClip
# from moviepy.audio.fx.all import audio_fadein, audio_fadeout

# # Function to extract only music from a video
# def extract_music(input_video_path, output_audio_path):
#     video_clip = VideoFileClip(input_video_path)
#     audio_clip = video_clip.audio
#     audio_clip.write_audiofile(output_audio_path, codec='mp3')

# # Function to apply extracted music to another video
# def apply_music_to_video(input_music_path, input_video_path, output_video_path):
#     # Load video clip without audio
#     video_clip = VideoFileClip(input_video_path)

#     # Debug: Print video metadata
#     print("Video Metadata:", video_clip.reader.infos)

#     # Load music clip
#     music_clip = AudioFileClip(input_music_path)

#     # Set the audio of the video clip to the loaded music clip
#     video_clip = video_clip.set_audio(music_clip)

#     # Optional: Fade-in and fade-out the audio
#     audio_clip = video_clip.audio.fx(audio_fadein, 2.0).fx(audio_fadeout, 2.0)
#     video_clip = video_clip.set_audio(audio_clip)

#     # Debug: Print final video metadata
#     print("Final Video Metadata:", video_clip.reader.infos)

#     # Write the final video with the merged audio
#     video_clip.write_videofile(output_video_path, codec='libx264', audio_codec='aac', fps=30)

# # Example usage
# input_video_path = "/content/drive/MyDrive/HP_short.mp4"
# output_audio_path = "/content/sample_data/extracted_music.mp3"
# input_music_path = "/content/sample_data/extracted_music.mp3"
# trailer_video_path = "/content/sample_data.trailer2.mp4"
# output_video_path = "/content/sample_data/output_video_with_music.mp4"

# # Extract music from the input video
# extract_music(input_video_path, output_audio_path)

# # Apply extracted music to another video
# apply_music_to_video(output_audio_path, trailer_video_path, output_video_path)


MoviePy - Writing audio in /content/sample_data/extracted_music.mp3


MoviePy - Done.
Video Metadata: {'duration': 14.13, 'video_found': True, 'video_size': [852, 480], 'video_fps': 30.0, 'video_nframes': 424, 'video_duration': 14.13, 'video_rotation': 0, 'audio_found': False}
Final Video Metadata: {'duration': 14.13, 'video_found': True, 'video_size': [852, 480], 'video_fps': 30.0, 'video_nframes': 424, 'video_duration': 14.13, 'video_rotation': 0, 'audio_found': False}
Moviepy - Building video /content/sample_data/output_video_with_music.mp4.
MoviePy - Writing audio in output_video_with_musicTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/sample_data/output_video_with_music.mp4



Moviepy - Done !
Moviepy - video ready /content/sample_data/output_video_with_music.mp4


In [ ]:
pip install spleeter


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip uninstall tensorflow


Found existing installation: tensorflow 2.9.3
Uninstalling tensorflow-2.9.3:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.9.3.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? Y
  Successfully uninstalled tensorflow-2.9.3
ERROR: Could not find a version that satisfies the requirement tensorflow==2.3.0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0,

In [ ]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 576.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 58.1 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard-data-server
    F